In [43]:
using Random
using Distributions

include("simulate.jl")
include("data.jl")

generateData (generic function with 3 methods)

In [48]:
"""
DATA CONFIG
"""
alpha = 0.05

# data
B  = 1    # num. coverage probabilities per boxplot
S  = 1   # num. samples per coverage probability
nx = 25      # size of group 1
ny = 13     # size of group 2
dtype = Float32
seed = 123

"""
POPULATION SETTINGS
"""

Random.seed!(seed)
distrTypeX = LogNormal{dtype}
muX = dtype.(round.(rand(Uniform(0, 1), B), digits=4))
sdX = 2 * muX
paramsX = (muX, sdX)

distrTypeY = LogNormal{dtype}
muY = dtype.(round.(rand(Uniform(0, 1), B), digits=4))
sdY = 2 * muY
paramsY = (muY, sdY)

x, y, deltas, distrX, distrY = generateData(B, S, nx, ny,
                                            distrTypeX, paramsX, distrTypeY, paramsY,
                                            dtype, seed)

([0.8590659; 0.3663767; … ; 2.5245826; 1.886721;;;], [2.4790723; 1.922852; … ; 5.880738; 1.8599702;;;], Float32[-0.6809921], LogNormal{Float32}[LogNormal{Float32}(μ=0.5212f0, σ=1.0424f0)], LogNormal{Float32}[LogNormal{Float32}(μ=0.5868f0, σ=1.1736f0)])

In [44]:
mc_size = try
    (binomial(nx+ny, nx) > 45_000) ? 10_000 : 0
catch
    0
end

simulate_perm(x, y, deltas, distrX, distrY,
              alpha=alpha, mc_size=mc_size, save_csv=false)

LoadError: type Tuple has no field pooled

In [ ]:
simulate_bootstrap(x, y, deltas, distrX, distrY,
                   alpha=alpha, nsamples=10_000, save_csv=false)

In [ ]:
simulate_t(x, y, deltas, distrX, distrY,
           alpha=alpha, save_csv=false)